#Basic examples

###Make sure you load the API keys for cloud providers!

###You can set your environment keys yourself or use a script. Please note that since keys are private, they are not included in the repository.


In [4]:
import os
import sys

sys.path.insert(0, os.path.abspath(".."))

from config import set_environment

set_environment()

In [5]:
from langchain_openai import OpenAI
from langchain_google_genai import GoogleGenerativeAI

openai_llm = OpenAI()
google_llm = GoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20")

# response = openai_llm.invoke("Tell me a joke about light bulbs!")
response = google_llm.invoke("Tell me a joke about light bulbs!")
print(response)

Why did the light bulb get an award?
Because it was outstanding in its field! (Or, because it had so many bright ideas!)


###Development testing, FakeListLLM


In [8]:
from langchain_community.llms import FakeListLLM

fake_llm = FakeListLLM(responses=["Hello"])
result = fake_llm.invoke("Tell me a joke about light bulbs!")
print(result)

Hello


In [9]:
# from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# chat = ChatAnthropic(model_name="claude-3-5-sonnet-latest", timeout=None, stop=None)
chat = ChatOpenAI(model="gpt-4o")
messages = [
    SystemMessage("You're a helpful programming assistant"),
    HumanMessage("Write a Python function to calculate factorial."),
]
response = chat.invoke(messages)
print(response)

content='Certainly! You can calculate the factorial of a number using a simple recursive function or an iterative approach. Here\'s how you can implement both methods in Python:\n\n1. **Recursive approach**:\n\n```python\ndef factorial_recursive(n):\n    if n < 0:\n        raise ValueError("Factorial is not defined for negative numbers")\n    if n == 0 or n == 1:\n        return 1\n    else:\n        return n * factorial_recursive(n - 1)\n\n# Example usage:\nprint(factorial_recursive(5))  # Output: 120\n```\n\n2. **Iterative approach**:\n\n```python\ndef factorial_iterative(n):\n    if n < 0:\n        raise ValueError("Factorial is not defined for negative numbers")\n    result = 1\n    for i in range(2, n + 1):\n        result *= i\n    return result\n\n# Example usage:\nprint(factorial_iterative(5))  # Output: 120\n```\n\nBoth functions will calculate the factorial of a given non-negative integer `n`. Please note that factorial is only defined for non-negative integers, so it\'s good

### Openai with thinking


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a problem-solving assistant."),
        ("user", "{ problem}"),
    ]
)

chat = ChatOpenAI(model="o3-mini", reasoning_effort="high")
chain = template | chat
response = chain.invoke({"problem": "Calculate the optimal strategy for..."})

chat = ChatOpenAI(model="gpt-4o")
chain = template | chat
response = chain.invoke({"problem": "Calculate the optimal strategy for..."})

### Simple workflows with LCEL


In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Create components
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
llm = ChatOpenAI()
output_parser = StrOutputParser()

# Chain them together using LCEL
chain = prompt | llm | output_parser
# Execute the workflow with a single call
result = chain.invoke({"topic": "programming"})
print(result)

Why do programmers prefer dark mode? 

Because light attracts bugs!


### Complex chain example


In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# Initialize the model
llm = GoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20")

# First chain generates s story
story_prompt = PromptTemplate.from_template("Write a short story about {topic}")
story_chain = story_prompt | llm | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analyze the following story's mood:\n{story}"
)
analysis_chain = analysis_prompt | llm | StrOutputParser()

# Combine chains
story_with_analysis = story_chain | analysis_chain

# Run the combined chain
story_analysis = story_with_analysis.invoke({"topic": "a rainy day"})
print(story_analysis)

The mood of this story is overwhelmingly **peaceful, serene, and introspective**, with strong undertones of **comfort, contentment, and quiet restoration.**

Here's a breakdown of how this mood is established:

1.  **Sensory Details Emphasizing Gentleness and Softness:**
    *   **Visuals:** "watercolor blur," "shades of grey and muted green," "tiny rivers formed on the glass, merging and separating," "light outside softened even further, a deep twilight." These images are soft, calming, and inviting to contemplation rather than action.
    *   **Sounds:** The rain is described as a "steady drumming," a "rhythmic hiss," a "persistent, gentle presence," and "nature's soft lullaby." The "relentless hum of the city" fades, replaced by this soothing natural soundscape. The kettle "whistled a soft tune." The rain is a "steady, comforting murmur," and the world "whisper[s]." The absence of harsh or demanding sounds is key.
    *   **Smell:** "faint, clean scent of damp earth and old books" e